In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reamostrando a imagem

In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.5 MB/s eta 0:00:00


In [3]:
import rasterio
from rasterio.enums import Resampling

# Caminhos para os arquivos
path_vnir = '/content/drive/MyDrive/OmanChromiteResearch/WorldView/015961633010_01_P002_MUL/21MAY23065945-M2AS-015961633010_01_P002_ACOMP.TIF'
path_swir = '/content/drive/MyDrive/OmanChromiteResearch/WorldView/015961633010_01_P001_SWR/21MAY23065946-A2AS-015961633010_01_P001_ACOMP.TIF'

# Abrir os datasets
with rasterio.open(path_vnir) as vnir, rasterio.open(path_swir) as swir:

    # Calcular a resolução da imagem SWIR
    swir_res = (swir.transform[0], -swir.transform[4])

    # Novos metadados
    new_meta = vnir.meta.copy()
    new_meta.update({
        'transform': vnir.transform * vnir.transform.scale((vnir.width / swir.width), (vnir.height / swir.height)),
        'width': swir.width,
        'height': swir.height,
    })

    # Reamostrar VNIR para a resolução de SWIR
    with rasterio.open('vnir_resampled.tif', 'w', **new_meta) as vnir_resampled:
        vnir_resampled.write(vnir.read(
            out_shape=(vnir.count, swir.height, swir.width),
            resampling=Resampling.bilinear
        ))


In [4]:
import numpy as np

# Abrir as imagens reamostrada e SWIR
with rasterio.open('vnir_resampled.tif') as vnir_resampled, rasterio.open(path_swir) as swir:

    # Novos metadados
    new_meta = vnir_resampled.meta.copy()
    new_meta.update(count=vnir_resampled.count + swir.count)  # Atualizar o número de bandas

    # Escrever novo arquivo com bandas concatenadas
    with rasterio.open('stacked_image.tif', 'w', **new_meta) as stacked:
        # Escrever bandas de VNIR
        for idx in range(vnir_resampled.count):
            stacked.write(vnir_resampled.read(idx + 1), idx + 1)

        # Escrever bandas de SWIR
        for idx in range(swir.count):
            stacked.write(swir.read(idx + 1), vnir_resampled.count + idx + 1)


In [5]:
path = '/content/stacked_image.tif'

In [6]:
# Abrir o arquivo GeoTIFF
with rasterio.open(path) as dataset:
    # Imprimir metadados gerais
    print(f"Driver: {dataset.driver}")
    print(f"Tamanho da imagem: {dataset.width}x{dataset.height}x{dataset.count}")
    print(f"Sistema de coordenadas: {dataset.crs}")
    print(f"Transformação: {dataset.transform}")

    # Metadados adicionais
    metadata = dataset.meta
    print("Metadados adicionais:")
    for key, value in metadata.items():
        print(f"  {key}: {value}")

    # Informações sobre as bandas
    for i in range(dataset.count):
        band = dataset.read(i + 1)
        print(f"Banda {i + 1}:")
        print(f"  Tipo de dado: {dataset.dtypes[i]}")
        print(f"  Valor mínimo: {band.min()}")
        print(f"  Valor máximo: {band.max()}")
        print(f"  NoData Value: {dataset.nodatavals[i]}")

    # Outros metadados
    tags = dataset.tags()
    if tags:
        print("Outros metadados:")
        for key, value in tags.items():
            print(f"  {key}: {value}")


Driver: GTiff
Tamanho da imagem: 3013x2523x16
Sistema de coordenadas: EPSG:32640
Transformação: | 3.70, 0.00, 416113.20|
| 0.00,-3.70, 2712730.80|
| 0.00, 0.00, 1.00|
Metadados adicionais:
  driver: GTiff
  dtype: uint16
  nodata: 0.0
  width: 3013
  height: 2523
  count: 16
  crs: EPSG:32640
  transform: | 3.70, 0.00, 416113.20|
| 0.00,-3.70, 2712730.80|
| 0.00, 0.00, 1.00|
Banda 1:
  Tipo de dado: uint16
  Valor mínimo: 0
  Valor máximo: 5810
  NoData Value: 0.0
Banda 2:
  Tipo de dado: uint16
  Valor mínimo: 0
  Valor máximo: 6594
  NoData Value: 0.0
Banda 3:
  Tipo de dado: uint16
  Valor mínimo: 0
  Valor máximo: 6976
  NoData Value: 0.0
Banda 4:
  Tipo de dado: uint16
  Valor mínimo: 0
  Valor máximo: 7505
  NoData Value: 0.0
Banda 5:
  Tipo de dado: uint16
  Valor mínimo: 0
  Valor máximo: 7863
  NoData Value: 0.0
Banda 6:
  Tipo de dado: uint16
  Valor mínimo: 0
  Valor máximo: 8327
  NoData Value: 0.0
Banda 7:
  Tipo de dado: uint16
  Valor mínimo: 0
  Valor máximo: 8185
  NoD